# 作業

以正確的副檔名下載網頁中的圖片
範例網頁:https://www.ptt.cc/bbs/Beauty/M.1556291059.A.75A.html

Note：因為 PTT 會詢問「是否滿 18 歲」，這邊可以用 cookies 繞過

`requests.get(URL, cookies={'over18': '1'}`


In [1]:
import requests
import os
import io
from bs4 import BeautifulSoup
from PIL import Image # pip install Pillow

def download_file(save_dir, image_id, url):
    local_filename = url.split('/')[-1]
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        # 另一種計算圖檔大小方式
        # image_b = io.BytesIO(r.content).read()
        # size = len(image_b)

        img = Image.open(r.raw)
        size = len(img.fp.read())
        
        # 檢查圖片副檔名
        pic_type = img.format
        pic_size = img.size
        pic_mode = img.mode
        img_savename = '{}/{}.{}'.format(save_dir, image_id, pic_type)

        if (size/1048576) <= 1:
            img.save(img_savename)
        else:
            with requests.get(url, stream=True) as r:
                r.raise_for_status()
                with open(img_savename, 'wb') as f:
                    for chunk in r.iter_content(chunk_size=8192): 
                        if chunk: 
                            f.write(chunk)
            return local_filename

In [2]:
url = 'https://www.ptt.cc/bbs/Beauty/M.1556291059.A.75A.html'
resp = requests.get(url, cookies={'over18': '1'})
soup = BeautifulSoup(resp.text)


In [3]:
# 決定要儲存的資料夾
output_dir = 'D:/learning/2nd-PyCrawlerMarathon/teach/D09/pic'

# 假如資料夾不存在就新增一個資料夾
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 定位所有圖片的 tag
url_keyword = 'imgur'

image_tags = soup.find(id='main-content').findChildren('a', recursive=False)
for img_tag in image_tags:
    # 取得所有圖片在第三方服務的 id
    # 加入if-else判別機制排除非網址
    # 組合圖片而非網站的網址
    if url_keyword not in img_tag['href']:
        print('not url')
        continue 
    else:
        image_id = img_tag['href'].split('/')[-1]
        pic_url = 'https://i.imgur.com/{}.jpg'.format(image_id)
    # 對圖片送出請求
        download_file(output_dir, image_id, pic_url)
        # download_file_big(pic_url)
    